### RNN, 어텐션 기법을 활용한 자연어 처리
자연어 문제를 위해 많이 사용하는 방법은 순환 신경망입니다. 문장에서 다음 글자를 예측하도록 훈련하는 문자 단위 RNN부터, 새로운 텍스트를 생성하고 그 과정에서 매우 긴 시퀀스를 가진 텐서플로 데이터셋을 만드는 방법을 알아봅니다.

### Char-RNN을 사용해 셰익스피어 문체 생성하기
Char-RNN을 이용해 한 번에 한 글자씩 새로운 텍스트를 생성할 수 있습니다. 데이터셋부터 구축합니다.

In [1]:
from keras.utils import get_file

shakespeare_url = "https://homl.info/shakespeare"
filepath = get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

데이터셋을 불러왔으니, 모든 글자를 정수로 인코딩해야 합니다. 케라스에서 제공하는 Tokenizer 클래스를 사용하여 이 클래스의 객체를 텍스트에 훈련시키고, 텍스트에서 사용되는 모든 글자를 찾아 각기 다른 글자 ID에 매핑합니다. 이 ID는 1부터 시작해 고유한 글자 개수까지 만들어집니다.

In [2]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(char_level = True)
tokenizer.fit_on_texts(shakespeare_text)

`char_level = True`로 지정하여 단어 수준 인코딩 대신 글자 수준 인코딩을 만듭니다. 이 클래스는 기본적으로 텍스트를 소문자로 바꿉니다. 이를 원하지 않을 경우 lower = False로 지정할 수 있습니다. 이제 문장을 글자 ID로 인코딩하거나 반대로 디코딩할 수 있습니다. 이를 통해 텍스트에 있는 고유 글자 개수와 전체 글자 개수를 알 수 있습니다.

In [3]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [4]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [5]:
max_id = len(tokenizer.word_index)       # 고유 글자 개수
dataset_size = tokenizer.document_count  # 전체 글자 개수

print(max_id, dataset_size)

39 1115394


전체 텍스트를 인코딩하여 각 글자를 Id로 나타내봅니다.

In [6]:
import numpy as np

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1 # 1에서 39까지 대신 0에서 38까지의 ID를 얻기 위해 1을 뺍니다.
[encoded]

[array([19,  5,  8, ..., 20, 26, 10])]

### 순차 데이터셋을 나누는 방법
훈련 세트, 검증 세트, 테스트 세트가 중복되지 않도록 만드는 것이 매우 중요합니다. 예를 들어서 텍스트의 처음 90%를 훈련 세트로 사용하고 다음 5%를 검증 세트, 마지막 5%를 테스트 세트로 사용할 수 있습니다. 두 세트 사이에 문장이 걸치지 않고 완전히 분리될 수 있도록 세트 사이에 간격을 두는 것도 좋은 생각입니다.

간단하게 얘기해서, 시계열을 훈련 세트, 테스트 세트, 검증 세트로 나누는 것은 간단한 작업이 아니고, 어떻게 나눌지 주어진 문제에 따라 달라집니다.

셰익스피어 데이터셋에 대해서는 텍스트의 처음 90%를 훈련 세트로 사용합니다. 나머지는 검증 세트와 테스트 세트로 활용하며, 이 세트에서 한 번에 한 글자씩 반환하는 `tf.data.Dataset`객체를 생성합니다.

In [7]:
import tensorflow as tf

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### 순차 데이터셋을 여러 개의 윈도우로 자르기
훈련 세트는 백만 개 이상의 글자로 이루어진 시퀀스 `하나`입니다. 여기에 신경망을 직접 훈련시킬 수는 없습니다. 이 RNN은 백만 개의 층이 있는 심층 신경망과 비슷하고 샘플 하나로 훈련하는 셈이 됩니다. 대신 데이터셋의 `window()`메소드를 사용해 이 긴 시퀀스를 작은, 많은 텍스트 윈도우로 변환합니다. 

이 데이터셋의 각 샘플은 전체 텍스트에서 매우 짧은 부분의 문자열입니다. RNN은 이 부분 문자열 길이만큼만 역전파를 수행하기 위해 펼쳐집니다. 이를 `TBTT(Truncated Backpropagation Trough Time)`라고 합니다. window() 메소드를 호출하여 짧은 텍스트 윈도우를 갖는 데이터셋을 만듭니다.

In [8]:
n_steps = 100 # n_steps도 튜닝 고려 사항 중 하나입니다. 짧은 입력 시퀀스에서 RNN을 훈련하는 것은 쉬빚만, 당연히 이 RNN은 n_steps보다 긴 패턴을 학습할 수 없습니다.
              # 따라서 너무 짧게 만들어서는 안되겠습니다.
              
window_length = n_steps + 1 # target = 1글자 앞의 input
dataset = dataset.window(window_length, shift = 1, drop_remainder = True)
len(dataset)

1003754

기본적으로 window() 메소드는 윈도우를 중복하지 않습니다. (파라미터에 대한 세부적인 내용은 아래 링크를 참고할 것)

> https://techblog-history-younghunjo1.tistory.com/373

shift = 1로 지정하면 가장 큰 훈련 세트를 만들 수 있습니다. 첫 번째 윈도우는 0에서 100번째 글자를 포함하고, 두 번째 윈도우는 1에서 101번째 글자를 포함하는 식입니다. (패딩 없이 데이터를 만들기 위해) 모든 윈도우가 동일하게 101개의 글자를 포함하도록 `drop_remainder = True`로 지정합니다. 그렇지 않으면 윈도우 100개는 글자 100개, 글자 99개와 같은 식으로 점점 줄어들어 마지막 윈도우는 글자 1개를 포함합니다.

window() 메소드는 각각 하나의 데이터셋으로 표현되는 윈도우를 포함하는 데이터셋을 만듭니다. 리스트의 리스트와 비슷한 `중첩 데이터셋`입니다. 이런 구조는 데이터셋 메소드를 호출하여 각 윈도우를 섞거나, 배치화 할 때 유용합니다. 하지만 모델은 데이터셋이 아니라 텐서를 기대하기 때문에 훈련에 중첩 데이터셋을 바로 사용할 수는 없습니다. 따라서 중첩 데이터셋을 `플랫 데이터셋`으로 변환하는 `flat_map()` 메소드를 호출해야 합니다. 

예를 들어 {1, 2, 3}이 텐서 1, 2, 3의 시퀀스를 포함한 데이터셋이라 가정하면, 중첩 데이터셋 {{1, 2}, {3, 4, 5, 6}}을 평평하게 만들면 플랫 데이터셋인 {1, 2, 3, 4, 5, 6}이 됩니다. flat_map() 메소드는 중첩 데이터셋을 평평하게 만들기 전에 각 데이터셋에 적용할 변환 함수를 매개변수로 받을 수 있습니다. 

예를 들어 lambda ds : ds.batch(2) 함수를 flat_map()에 전달하면 중첩 데이터셋 {{1, 2}, {3, 4, 5, 6}}을 플랫 데이터셋 {{1, 2}, {3, 4}, {5, 6}}으로 변환합니다. 이는 텐서 2개를 가진 데이터셋입니다.

In [9]:
dataset = dataset.flat_map(lambda window : window.batch(window_length))

위 데이터셋은 batch(window_length)를 호출합니다. 이 길이는 윈도우 길이와 같이 대문에 텐서 하나를 담은 데이터셋을 얻습니다. 이 데이터셋은 연속된 101글자 길이의 운도우를 담습니다. 경사 하강법은 훈련 세트에서 샘플이 동일 독립 분포일 때 가장 잘 작동하기 때문에 이 윈도우를 섞어야 합니다. 그 다음 윈도우를 배치로 만들고 입력(최초 100개의 글자)과 타겟(마지막 100개의 글자)을 분리하겠습니다.

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows : (windows[:, :-1], windows[:, 1:]))

지금까지 진행한 데이터 전처리 과정을 그림으로 나타내면 아래와 같습니다.

<div align=center><img src="../screenshots/preteate_txt.jpg"></div>

일반적으로 범주형 입력 특성은 원-핫 벡터나 임베딩으로 인코딩 되어야 합니다. 여기에서는 고유한 글자 수가 적기 때문에 원-핫 벡터를 사용해 글자를 인코딩합니다.

OneHotEncoding의 문제점
- 시퀀스의 양이 많을 수록, 공간이 계속 늘어난다
    + 예를 들어, 5개의 악기라는 카테고리를 기반으로 OneHotEncoding을 진행하면 상관없지만, 만약 종류가 10000개라면 1개만 1이고, 9999개의 0으로 구성된 벡터로 표현되므로 이러한 0들은 희소 벡터가 됨과 동시에 벡터를 저장하기 위해 필요한 공간이 계속 늘어나므로 공간의 낭비가 발생하고 컴퓨터의 성능을 저하시킨다.
- 텍스트 유사도를 표현할 수 없다.
    + 각 텍스트 간에 부여받은 원핫 벡터로는 유사도를 책정할 수 없다. 단어 간 유사성을 파악할 수 없다는 것은 검색 시스템을 구현할 때 문제가 발생한다. 

이러한 단점들을 보완한 벡터화 방법이 많이 있다.
- 카운트 기반의 벡터화 방법인 `LSA(잠재 의미 분석)`
- 예측 기반으로 벡터화 하는 `NNLM`, `RNNLM`, `Word2Vec`, `FastText`
- 위의 두 가지 방법을 모두 사용하는 `GloVe`

In [11]:
# 원핫 인코딩 진행
dataset = dataset.map(lambda X_batch, Y_batch : (tf.one_hot(X_batch, depth = max_id), Y_batch))

마지막으로 data에 prefetch를 적용해야 합니다.

##### prefetch ?
- 훈련 속도를 더 빠르게 하기 위해 사용한다.
- `prefetch(1)`을 호출하면 데이터셋은 항상 한 배치가 미리 준비된다.(알고리즘이 한 배치로 작업하는 동안에 다음 배치가 준비되는 식)
- GPU에서 훈련하는 스텝을 수행하는 것보다 짧은 시간 안에 한 배치 데이터를 준비할 수 있다.

### Char-RNN 모델 만들고 훈련하기
이전 글자 100개를 기반으로 다음 글자를 예측하기 위해 유닛 128개를 가진 GRU층 2개와 입력(dropout)과 은닉 상태(recurrent_dropout)에 20% 드롭아웃을 사용합니다. 필요하면 나중에 이 하이퍼 파라미터를 수정할 수 있습니다. 출력층은 TimeDistributed클래스를 적용한 Dense층입니다. 

텍스트에 있는 고유한 글자 수는 39개이므로 이 층은 39개의 유닛을 가져야 합니다. 타임 스텝마다 각 글자에 대한 확률을 출력할 수 있습니다. 타입 스텝에서 출력 확률의 합은 1이어야 하므로 Denses층의 출력에 Softmax 함수를 적용합니다. 그 다음 원핫 인코딩된 시퀀스이기에 손실함수로 sparse_categorical_crossentropy를 사용하고 Adam 옵티마이저를 적용해 model을 컴파일합니다. 

In [23]:
from keras.models import Sequential
from keras.layers import GRU, Dense, TimeDistributed

model = Sequential()
model.add(GRU(128, return_sequences = True, input_shape = [None, max_id], dropout = 0.2, recurrent_dropout = 0, 
              activation = "tanh", recurrent_activation = "sigmoid", unroll = False, use_bias = True, reset_after = True))
model.add(GRU(128, return_sequences = True, dropout = 0.2, recurrent_dropout = 0, 
              activation = "tanh", recurrent_activation = "sigmoid", unroll = False, use_bias = True, reset_after = True))
model.add(TimeDistributed(Dense(max_id, activation = "softmax")))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["acc"])

In [24]:
history = model.fit(dataset, epochs = 20)

Epoch 1/20
31368/31368 [==============================] - 531s 17ms/step - loss: 1.6208 - acc: 0.5049
Epoch 2/20
31368/31368 [==============================] - 532s 17ms/step - loss: 1.5387 - acc: 0.5253
Epoch 3/20
31368/31368 [==============================] - 494s 16ms/step - loss: 1.5166 - acc: 0.5310
Epoch 4/20
31368/31368 [==============================] - 470s 15ms/step - loss: 1.5043 - acc: 0.5341
Epoch 5/20
31368/31368 [==============================] - 459s 15ms/step - loss: 1.4963 - acc: 0.5361
Epoch 6/20
31368/31368 [==============================] - 457s 15ms/step - loss: 1.4914 - acc: 0.5374
Epoch 7/20
31368/31368 [==============================] - 456s 15ms/step - loss: 1.4872 - acc: 0.5384
Epoch 8/20
31368/31368 [==============================] - 456s 14ms/step - loss: 1.4838 - acc: 0.5393
Epoch 9/20
31368/31368 [==============================] - 528s 17ms/step - loss: 1.4814 - acc: 0.5399
Epoch 10/20
31368/31368 [==============================] - 573s 18ms/step - loss: 

### 모델 사용하기
모델이 완성되었습니다. 이 모델에 새로운 텍스트를 주입하려면 앞에서와 같이 먼저 전처리를 진행해야 합니다.

In [28]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

X_new = preprocess(["How are yo"])
Y_pred = np.argmax(model(X_new), axis = 1)
tokenizer.sequences_to_texts(Y_pred)[0][-1]

' '